In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
pd.options.display.max_rows = 999

## 데이터 확인하기

In [2]:
url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev'
key = 'z6guLe%2B1ADHJXivIOzOUKTwnkcYgHhhB0haRkPGqJzTu3tF1oebdDMG%2BoIvRnSv2eUJ6a%2FxGly%2BqbR3TKc9Skg%3D%3D'
queryParams = '?'+'LAWD_CD='+'11110'+'&ServiceKey='+ key+'&DEAL_YMD='+'201904'
url = url + queryParams
url

'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?LAWD_CD=11110&ServiceKey=z6guLe%2B1ADHJXivIOzOUKTwnkcYgHhhB0haRkPGqJzTu3tF1oebdDMG%2BoIvRnSv2eUJ6a%2FxGly%2BqbR3TKc9Skg%3D%3D&DEAL_YMD=201904'

In [3]:
result = requests.get(url)

In [4]:
bs_obj=BeautifulSoup(result.content, "lxml-xml")
#파싱한 컨텐츠를 들여쓰기 하여 표현
print(bs_obj.prettify())

<?xml version="1.0" encoding="utf-8"?>
<response>
 <header>
  <resultCode>
   99
  </resultCode>
  <resultMsg>
   LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.
  </resultMsg>
 </header>
</response>


In [22]:
cols=['지역코드','법정동','년','월','일','아파트','건축년도','층','전용면적','거래금액']

## 법정동 시군구 코드 가져오기

In [23]:
path ='./data/법정동코드 전체자료.txt'

In [24]:
LAWD_CD = pd.read_csv(path, delimiter='\t', encoding='euc-kr')
LAWD_CD.head()

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재


In [25]:
LAWD_CD['법정동시군구코드']=LAWD_CD['법정동코드'].apply(lambda x: str(x)[:5])
LAWD_CD['시군구']=LAWD_CD['법정동명'].apply(lambda x: x.split(' ')[:2])
LAWD_CD.head()

,법정동코드,법정동명,폐지여부,법정동시군구코드,시군구
0,1100000000,서울특별시,존재,11000,[서울특별시]
1,1111000000,서울특별시 종로구,존재,11110,"[서울특별시, 종로구]"
2,1111010100,서울특별시 종로구 청운동,존재,11110,"[서울특별시, 종로구]"
3,1111010200,서울특별시 종로구 신교동,존재,11110,"[서울특별시, 종로구]"
4,1111010300,서울특별시 종로구 궁정동,존재,11110,"[서울특별시, 종로구]"


In [26]:
LAWD_CD_SEOUL=LAWD_CD.loc[LAWD_CD['법정동명'].str.contains('서울특별시')]
LAWD_CODE_SEOUL=LAWD_CD_SEOUL['법정동시군구코드'].unique()[1:]
print(LAWD_CODE_SEOUL.shape)
LAWD_CODE_SEOUL

(25,)


array(['11110', '11140', '11170', '11200', '11215', '11230', '11260',
       '11290', '11305', '11320', '11350', '11380', '11410', '11440',
       '11470', '11500', '11530', '11545', '11560', '11590', '11620',
       '11650', '11680', '11710', '11740'], dtype=object)

### 월별 실거래가 모듈

In [8]:
df_month = pd.DataFrame(columns=cols)
for month in list(range(1,13)):
    if month < 10:
        month = '0'+ str(month)
    else:
        month = str(month)
    
    url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev'
    key = 'z6guLe%2B1ADHJXivIOzOUKTwnkcYgHhhB0haRkPGqJzTu3tF1oebdDMG%2BoIvRnSv2eUJ6a%2FxGly%2BqbR3TKc9Skg'
    queryParams = '?'+'LAWD_CD='+'11110'+'&ServiceKey='+ key+'&DEAL_YMD='+'2018'+ month
    url = url + queryParams
    
    result = requests.get(url)
    bs_obj=BeautifulSoup(result.content, "lxml-xml")
    items = bs_obj.findAll('item')
    

    for i, item in enumerate(items):
        df = pd.DataFrame(columns=cols)
        df.loc[i, "거래금액"] = item.find("거래금액").text
        df.loc[i, "건축년도"] = item.find("건축년도").text
        df.loc[i, "아파트"] = item.find("아파트").text
        df.loc[i, "법정동"] = item.find("법정동").text
        df.loc[i, "년"] = item.find("년").text
        df.loc[i, "월"] = item.find("월").text
        df.loc[i, "일"] = item.find("일").text
        df.loc[i, "전용면적"] = item.find("전용면적").text
        df.loc[i, "지역코드"] = item.find("지역코드").text
        df.loc[i, "층"] = item.find("층").text
        
        df_month = pd.concat([df_month, df], axis=0)

print(df_month.shape)


(0, 10)


In [9]:
df_month.head()

,지역코드,법정동,년,월,일,아파트,건축년도,층,전용면적,거래금액


### 연도별 모듈

In [44]:
y1=2014
y2=2018

In [ ]:
df_all = pd.DataFrame(columns=cols)
for year in list(range(y1,y2)):
    year=str(year)
    for month in list(range(1,13)):
        if month < 10:
            month = '0'+ str(month)
        else:
            month = str(month)

        url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev'
        key = 'z6guLe%2B1ADHJXivIOzOUKTwnkcYgHhhB0haRkPGqJzTu3tF1oebdDMG%2BoIvRnSv2eUJ6a%2FxGly%2BqbR3TKc9Skg'
        queryParams = '?'+'LAWD_CD='+'11110'+'&ServiceKey='+ key+'&DEAL_YMD='+year+ month
        url = url + queryParams

        result = requests.get(url)
        bs_obj=BeautifulSoup(result.content, "lxml-xml")
        items = bs_obj.findAll('item')


        for i, item in enumerate(items):
            df = pd.DataFrame(columns=cols)
            df.loc[i, "거래금액"] = item.find("거래금액").text
            df.loc[i, "건축년도"] = item.find("건축년도").text
            df.loc[i, "아파트"] = item.find("아파트").text
            df.loc[i, "년"] = item.find("년").text
            df.loc[i, "월"] = item.find("월").text
            df.loc[i, "일"] = item.find("일").text
            df.loc[i, "전용면적"] = item.find("전용면적").text
            df.loc[i, "지역코드"] = item.find("지역코드").text
            df.loc[i, "층"] = item.find("층").text
            df.loc[i, "법정동"] = item.find("법정동").text


            df_all = pd.concat([df_all, df], axis=0)

print(df_all.shape)

In [ ]:
df_all.head()

### 지역코드(법정동시군구코드) 모듈

In [45]:
df_all = pd.DataFrame(columns=cols)
    
for LAWD_CD_seoul in LAWD_CODE_SEOUL:
    LAWD_CD_seoul=str(LAWD_CD_seoul)
    
    for year in list(range(y1,y2)):
        year=str(year)
        
        for month in list(range(1,13)):
            if month < 10:
                month = '0'+ str(month)
            else:
                month = str(month)

            url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev'
            key = 'z6guLe%2B1ADHJXivIOzOUKTwnkcYgHhhB0haRkPGqJzTu3tF1oebdDMG%2BoIvRnSv2eUJ6a%2FxGly%2BqbR3TKc9Skg%3D%3D'
            queryParams = '?' + 'LAWD_CD=' + LAWD_CD_seoul + '&ServiceKey='+ key + '&DEAL_YMD=' + year + month
            url = url + queryParams

            result = requests.get(url)
            bs_obj=BeautifulSoup(result.content, "lxml-xml")
            items = bs_obj.findAll('item')


            for i, item in enumerate(items):
                df = pd.DataFrame(columns=cols)
                df.loc[i, "거래금액"] = item.find("거래금액").text
                df.loc[i, "건축년도"] = item.find("건축년도").text
                df.loc[i, "아파트"] = item.find("아파트").text
                df.loc[i, "년"] = item.find("년").text
                df.loc[i, "월"] = item.find("월").text
                df.loc[i, "일"] = item.find("일").text
                df.loc[i, "전용면적"] = item.find("전용면적").text
                df.loc[i, "지역코드"] = item.find("지역코드").text
                df.loc[i, "층"] = item.find("층").text
                df.loc[i, "법정동"] = item.find("법정동").text


                df_all = pd.concat([df_all, df], axis=0)

print(df_all.shape)

(0, 10)


In [46]:
df_all.head()

,지역코드,법정동,년,월,일,아파트,건축년도,층,전용면적,거래금액


In [50]:
df_all.to_csv('./data/서울시_아파트_실거래가_{}~{}.csv'.format(y1, y2))